# Modelos de predicción de robos a Casa Habitación

In [ ]:
import pandas as pd
import numpy as np

# 1. Cargar el archivo de promedios
file_path_casa = "exportados/Robos a casa habitacion/cuadrantes_robos_CaHa_promedio.xlsx"
df_promedios_casa = pd.read_excel(file_path_casa)

# Renombrar columnas para facilitar el manejo
column_names_casa = {
    'CUADRANTE': 'CUADRANTE',
    'POBLACION': 'POBLACION'
}
for i in range(1, 13):
    column_names_casa[f'PROMEDIO DE ROBOS A CASA HABITACION MES {i}'] = f'ROBOS_MES_{i}'

df_promedios_casa.rename(columns=column_names_casa, inplace=True)

# 2. Desapilar (Unpivot) la tabla para crear las 936 filas
# La columna 'POBLACION' se mantiene fija para cada cuadrante.
df_long_casa = df_promedios_casa.melt(
    id_vars=['CUADRANTE', 'POBLACION'],
    value_vars=[f'ROBOS_MES_{i}' for i in range(1, 13)],
    var_name='MES_NOMBRE',
    value_name='ROBOS_MES_N' # Este es el valor que queremos predecir (Y)
)

# 3. Extraer el número de mes (MES_N)
df_long_casa['MES_N'] = df_long_casa['MES_NOMBRE'].str.extract(r'(\d+)').astype(int)
df_long_casa.drop(columns=['MES_NOMBRE'], inplace=True)

# 4. Ordenar la tabla por CUADRANTE y MES_N
df_long_casa.sort_values(by=['MES_N', 'CUADRANTE'], inplace=True)
df_long_casa.reset_index(drop=True, inplace=True)

print("### 1. DataFrame de 936 Filas (Formato de Serie de Tiempo) ###")
print(df_long_casa.head(80).tail(5)) # Muestra los últimos del Mes 1
print(f"\nTotal de filas: {len(df_long_casa)}")
print("-" * 50)

In [ ]:
# 5. Crear la columna de Diciembre (MES 12) para usarla como N-1 del Mes 1
# Extraer los promedios de Diciembre (MES 12)
df_diciembre_casa = df_long_casa[df_long_casa['MES_N'] == 12].copy()
df_diciembre_casa.rename(columns={'ROBOS_MES_N': 'ROBOS_DIC'}, inplace=True)
df_diciembre_casa = df_diciembre_casa[['CUADRANTE', 'ROBOS_DIC']]

# Extraer los promedios de Noviembre (MES 11)
df_noviembre_casa = df_long_casa[df_long_casa['MES_N'] == 11].copy()
df_noviembre_casa.rename(columns={'ROBOS_MES_N': 'ROBOS_NOV'}, inplace=True)
df_noviembre_casa = df_noviembre_casa[['CUADRANTE', 'ROBOS_NOV']]

# 6. Crear las columnas N-1 y N-2 usando 'shift' agrupado
df_long_casa['ROBOS_MES_N_MENOS_1'] = df_long_casa.groupby('CUADRANTE')['ROBOS_MES_N'].shift(1)
df_long_casa['ROBOS_MES_N_MENOS_2'] = df_long_casa.groupby('CUADRANTE')['ROBOS_MES_N'].shift(2)

# 7. Imputar Enero (MES_N = 1 y 2) con los valores de Diciembre y Noviembre
# El Mes 1 (Enero) toma N-1 de Diciembre y N-2 de Noviembre.
df_long_casa = df_long_casa.merge(df_diciembre_casa, on='CUADRANTE', how='left')
df_long_casa.loc[df_long_casa['MES_N'] == 1, 'ROBOS_MES_N_MENOS_1'] = df_long_casa.loc[df_long_casa['MES_N'] == 1, 'ROBOS_DIC']

df_long_casa = df_long_casa.merge(df_noviembre_casa, on='CUADRANTE', how='left')
df_long_casa.loc[df_long_casa['MES_N'] == 1, 'ROBOS_MES_N_MENOS_2'] = df_long_casa.loc[df_long_casa['MES_N'] == 1, 'ROBOS_NOV']
# El Mes 2 (Febrero) toma N-2 de Diciembre
df_long_casa.loc[df_long_casa['MES_N'] == 2, 'ROBOS_MES_N_MENOS_2'] = df_long_casa.loc[df_long_casa['MES_N'] == 2, 'ROBOS_DIC']

df_final_casa = df_long_casa.drop(columns=['ROBOS_DIC', 'ROBOS_NOV'])
print("### 2. DataFrame Final con Variables N-1 y N-2 (Primeros 5 cuadrantes) ###")
print(df_final_casa.head(5))
print("-" * 50)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler # <-- para normalizar

# 1. Separar la tabla para aplicar el escalador
df_scale_casa = df_final_casa[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2', 'ROBOS_MES_N']].copy()

# 2. Aplicar el escalador (entrenado SOLO con los datos de entrada X)
# El escalador debe tratar todas las variables por separado
scaler_casa = MinMaxScaler()
df_scaled_values_casa = scaler_casa.fit_transform(df_scale_casa)
df_scaled_casa = pd.DataFrame(df_scaled_values_casa, columns=df_scale_casa.columns)
df_scaled_casa['MES_N'] = df_final_casa['MES_N'].values # Añadir el MES_N de vuelta

# 3. Estructurar X y Y para Keras (Múltiples Salidas) con datos escalados
X_features_scaled_casa = df_scaled_casa[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']]
Y_output_scaled_casa = df_scaled_casa['ROBOS_MES_N']

# Agrupar los datos escalados
X_data_scaled_casa = X_features_scaled_casa.groupby(df_scaled_casa['MES_N']).apply(lambda x: x.values.flatten()).tolist()
Y_data_scaled_casa = Y_output_scaled_casa.groupby(df_scaled_casa['MES_N']).apply(lambda x: x.values).tolist()

X_scaled_casa = np.array(X_data_scaled_casa)
Y_scaled_casa = np.array(Y_data_scaled_casa)

# Dividir para entrenamiento y prueba (mantenemos test_size=0.15 por ahora)
X_train_scaled_casa, X_test_scaled_casa, Y_train_scaled_casa, Y_test_scaled_casa = train_test_split(
    X_scaled_casa, Y_scaled_casa, test_size=0.15, shuffle=False
)

input_dim_casa = X_scaled_casa.shape[1]
output_dim_casa = Y_scaled_casa.shape[1]
# 4. Reconstruir y Entrenar el Modelo Keras (con datos escalados)
# Se reduce la tasa de aprendizaje (learning rate) como medida de precaución adicional
model_scaled_casa = Sequential([
    Dense(512, activation='relu', input_shape=(input_dim_casa,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim_casa, activation='linear')
])

model_scaled_casa.compile(optimizer=Adam(learning_rate=0.0005), loss='mse') # Reducción de LR
early_stop_casa = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

print("### Entrenamiento del Modelo Keras (Datos Normalizados) ###")
model_scaled_casa.fit(
    X_train_scaled_casa, Y_train_scaled_casa,
    epochs=200, # Aumentamos épocas por si acaso
    batch_size=2,
    validation_data=(X_test_scaled_casa, Y_test_scaled_casa),
    callbacks=[early_stop_casa],
    verbose=0
)
print("Entrenamiento con normalización finalizado.")

# 5. Evaluación (R^2)
Y_pred_test_scaled_casa = model_scaled_casa.predict(X_test_scaled_casa)
r2_scaled_casa = r2_score(Y_test_scaled_casa.flatten(), Y_pred_test_scaled_casa.flatten())

print(f"\n### Coeficiente de Determinación (R^2) con Normalización: {r2_scaled_casa:.4f} ###")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error


def predecir_y_evaluar_top_cuadrantes_casa(mes_a_predecir_casa, df_historico_casa, modelo_casa, escalador_casa, df_datos_reales_casa):
    """
    Predice y evalúa el robo promedio a casa habitación para los 78 cuadrantes en un mes N.

    Args:
        mes_a_predecir (int): El número del mes que se quiere predecir (ej. 13).
        df_historico (pd.DataFrame): El DataFrame con los promedios históricos (df_final).
        modelo (tf.keras.Model): El modelo de Keras entrenado (model_scaled).
        escalador (sklearn.preprocessing.MinMaxScaler): El objeto MinMaxScaler.
        df_datos_reales (pd.DataFrame): DataFrame con la columna 'ROBOS_MES_N' real para ese mes N.

    Returns:
        tuple: (DataFrame del Top 10, R2, MAE)
    """

    print(f"--- Ejecutando Predicción y Evaluación para el Mes {mes_a_predecir_casa} ---")

    # 1. Definir meses históricos cíclicos
    mes_n_menos_1_casa = (mes_a_predecir_casa - 1)
    mes_n_menos_2_casa = (mes_a_predecir_casa - 2)
    mes_anterior_ciclo_casa = (mes_n_menos_1_casa - 1) % 12 + 1
    mes_anterior_2_ciclo_casa = (mes_n_menos_2_casa - 1) % 12 + 1

    # 2. Obtener los 78 datos de entrada de los meses N-1 y N-2
    df_n_menos_1_casa = df_historico_casa[df_historico_casa['MES_N'] == mes_anterior_ciclo_casa].copy()
    df_n_menos_2_casa = df_historico_casa[df_historico_casa['MES_N'] == mes_anterior_2_ciclo_casa].copy()

    # 3. Construir el DataFrame de Entrada (78 filas)
    df_prediccion_casa = df_historico_casa[df_historico_casa['MES_N'] == 1].copy()

    # Asegurar orden y rellenar las columnas de entrada
    df_prediccion_casa.sort_values(by='CUADRANTE', inplace=True)
    df_n_menos_1_casa.sort_values(by='CUADRANTE', inplace=True)
    df_n_menos_2_casa.sort_values(by='CUADRANTE', inplace=True)

    df_prediccion_casa['ROBOS_MES_N_MENOS_1'] = df_n_menos_1_casa['ROBOS_MES_N'].values
    df_prediccion_casa['ROBOS_MES_N_MENOS_2'] = df_n_menos_2_casa['ROBOS_MES_N'].values

    # 4. Preparar, Escalar y Aplanar la matriz X (312 entradas)
    X_pred_raw_casa = df_prediccion_casa[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']].values
    X_con_dummy_casa = np.hstack((X_pred_raw_casa, np.zeros((X_pred_raw_casa.shape[0], 1))))
    X_pred_escalado_completo_casa = escalador_casa.transform(X_con_dummy_casa)
    X_pred_final_casa = X_pred_escalado_completo_casa[:, :4].flatten().reshape(1, -1)

    # 5. Realizar la Predicción (Y_pred_scaled)
    Y_pred_scaled_casa = modelo_casa.predict(X_pred_final_casa)

    # 6. Desescalar la Predicción (Y_pred_desescalado)
    Y_pred_full_casa = np.hstack((X_pred_raw_casa, Y_pred_scaled_casa.T))
    Y_pred_desescalado_casa = escalador_casa.inverse_transform(Y_pred_full_casa)[:, -1]

    # 7. Generar el Top N (DataFrame de resultados)
    df_resultado_casa = pd.DataFrame({
        'CUADRANTE': df_prediccion_casa['CUADRANTE'].values,
        'PREDICCION_ROBOS_MES_N': Y_pred_desescalado_casa,
        'MES_PREDICHO': mes_a_predecir_casa
    })

    # 8. EVALUACIÓN Y COMPARACIÓN con Datos Reales
    # Aseguramos que el orden de los datos reales coincida con la predicción (por CUADRANTE)
    df_datos_reales_casa.sort_values(by='CUADRANTE', inplace=True)
    Y_real_casa = df_datos_reales_casa['ROBOS_MES_N'].values

    # Unir predicción y real para la visualización del Top N
    df_resultado_casa['ROBOS_REALES_MES_N'] = Y_real_casa

    # Cálculo de Métricas
    r2_casa = r2_score(Y_real_casa, Y_pred_desescalado_casa)
    mae_casa = mean_absolute_error(Y_real_casa, Y_pred_desescalado_casa)

    # Generar Top 10 con la comparación
    top_10_casa = df_resultado_casa.sort_values(by='PREDICCION_ROBOS_MES_N', ascending=False).head(10).round(2)

    return top_10_casa, r2_casa, mae_casa

# Modelo de predicción de robos a Negocios

In [ ]:
import pandas as pd
import numpy as np

# 1. Cargar el archivo de promedios
file_path_neg = "exportados/Robos a negocios/cuadrantes_robos_negocios_promedio.xlsx"
df_promedios_neg = pd.read_excel(file_path_neg)

# Renombrar columnas para facilitar el manejo
column_names_neg = {
    'CUADRANTE': 'CUADRANTE',
    'POBLACION': 'POBLACION'
}
for i in range(1, 13):
    column_names_neg[f'PROMEDIO DE ROBOS A NEGOCIOS MES {i}'] = f'ROBOS_MES_{i}'

df_promedios_neg.rename(columns=column_names_neg, inplace=True)

# 2. Desapilar (Unpivot) la tabla para crear las 936 filas
# La columna 'POBLACION' se mantiene fija para cada cuadrante.
df_long_neg = df_promedios_neg.melt(
    id_vars=['CUADRANTE', 'POBLACION'],
    value_vars=[f'ROBOS_MES_{i}' for i in range(1, 13)],
    var_name='MES_NOMBRE',
    value_name='ROBOS_MES_N' # Este es el valor que queremos predecir (Y)
)

# 3. Extraer el número de mes (MES_N)
df_long_neg['MES_N'] = df_long_neg['MES_NOMBRE'].str.extract(r'(\d+)').astype(int)
df_long_neg.drop(columns=['MES_NOMBRE'], inplace=True)

# 4. Ordenar la tabla por CUADRANTE y MES_N
df_long_neg.sort_values(by=['MES_N', 'CUADRANTE'], inplace=True)
df_long_neg.reset_index(drop=True, inplace=True)

print("### 1. DataFrame de 936 Filas (Formato de Serie de Tiempo) ###")
print(df_long_neg.head(80).tail(5)) # Muestra los últimos del Mes 1
print(f"\nTotal de filas: {len(df_long_neg)}")
print("-" * 50)

In [ ]:
# 5. Crear la columna de Diciembre (MES 12) para usarla como N-1 del Mes 1
# Extraer los promedios de Diciembre (MES 12)
df_diciembre_neg = df_long_neg[df_long_neg['MES_N'] == 12].copy()
df_diciembre_neg.rename(columns={'ROBOS_MES_N': 'ROBOS_DIC'}, inplace=True)
df_diciembre_neg = df_diciembre_neg[['CUADRANTE', 'ROBOS_DIC']]

# Extraer los promedios de Noviembre (MES 11)
df_noviembre_neg = df_long_neg[df_long_neg['MES_N'] == 11].copy()
df_noviembre_neg.rename(columns={'ROBOS_MES_N': 'ROBOS_NOV'}, inplace=True)
df_noviembre_neg = df_noviembre_neg[['CUADRANTE', 'ROBOS_NOV']]

# 6. Crear las columnas N-1 y N-2 usando 'shift' agrupado
df_long_neg['ROBOS_MES_N_MENOS_1'] = df_long_neg.groupby('CUADRANTE')['ROBOS_MES_N'].shift(1)
df_long_neg['ROBOS_MES_N_MENOS_2'] = df_long_neg.groupby('CUADRANTE')['ROBOS_MES_N'].shift(2)

# 7. Imputar Enero (MES_N = 1 y 2) con los valores de Diciembre y Noviembre
# El Mes 1 (Enero) toma N-1 de Diciembre y N-2 de Noviembre.
df_long_neg = df_long_neg.merge(df_diciembre_neg, on='CUADRANTE', how='left')
df_long_neg.loc[df_long_neg['MES_N'] == 1, 'ROBOS_MES_N_MENOS_1'] = df_long_neg.loc[df_long_neg['MES_N'] == 1, 'ROBOS_DIC']

df_long_neg = df_long_neg.merge(df_noviembre_neg, on='CUADRANTE', how='left')
df_long_neg.loc[df_long_neg['MES_N'] == 1, 'ROBOS_MES_N_MENOS_2'] = df_long_neg.loc[df_long_neg['MES_N'] == 1, 'ROBOS_NOV']
# El Mes 2 (Febrero) toma N-2 de Diciembre
df_long_neg.loc[df_long_neg['MES_N'] == 2, 'ROBOS_MES_N_MENOS_2'] = df_long_neg.loc[df_long_neg['MES_N'] == 2, 'ROBOS_DIC']

df_final_neg = df_long_neg.drop(columns=['ROBOS_DIC', 'ROBOS_NOV'])

print("### 2. DataFrame Final con Variables N-1 y N-2 (Primeros 5 cuadrantes) ###")
print(df_final_neg.head(5))
print("-" * 50)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler # <-- para normalizar

# 1. Separar la tabla para aplicar el escalador
df_scale_neg = df_final_neg[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2', 'ROBOS_MES_N']].copy()

# 2. Aplicar el escalador (entrenado SOLO con los datos de entrada X)
# El escalador debe tratar todas las variables por separado
scaler_neg = MinMaxScaler()
df_scaled_values_neg = scaler_neg.fit_transform(df_scale_neg)
df_scaled_neg = pd.DataFrame(df_scaled_values_neg, columns=df_scale_neg.columns)
df_scaled_neg['MES_N'] = df_final_neg['MES_N'].values # Añadir el MES_N de vuelta

# 3. Estructurar X y Y para Keras (Múltiples Salidas) con datos escalados
X_features_scaled_neg = df_scaled_neg[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']]
Y_output_scaled_neg = df_scaled_neg['ROBOS_MES_N']

# Agrupar los datos escalados
X_data_scaled_neg = X_features_scaled_neg.groupby(df_scaled_neg['MES_N']).apply(lambda x: x.values.flatten()).tolist()
Y_data_scaled_neg = Y_output_scaled_neg.groupby(df_scaled_neg['MES_N']).apply(lambda x: x.values).tolist()

X_scaled_neg = np.array(X_data_scaled_neg)
Y_scaled_neg = np.array(Y_data_scaled_neg)

# Dividir para entrenamiento y prueba (mantenemos test_size=0.15 por ahora)
X_train_scaled_neg, X_test_scaled_neg, Y_train_scaled_neg, Y_test_scaled_neg = train_test_split(
    X_scaled_neg, Y_scaled_neg, test_size=0.15, shuffle=False
)

input_dim_neg = X_scaled_neg.shape[1]
output_dim_neg = Y_scaled_neg.shape[1]

# 4. Reconstruir y Entrenar el Modelo Keras (con datos escalados)
# Se reduce la tasa de aprendizaje (learning rate) como medida de precaución adicional
model_scaled_neg = Sequential([
    Dense(512, activation='relu', input_shape=(input_dim_neg,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim_neg, activation='linear')
])

model_scaled_neg.compile(optimizer=Adam(learning_rate=0.0005), loss='mse') # Reducción de LR
early_stop_neg = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

print("### Entrenamiento del Modelo Keras (Datos Normalizados) ###")
model_scaled_neg.fit(
    X_train_scaled_neg, Y_train_scaled_neg,
    epochs=200, # Aumentamos épocas por si acaso
    batch_size=2,
    validation_data=(X_test_scaled_neg, Y_test_scaled_neg),
    callbacks=[early_stop_neg],
    verbose=0
)
print("Entrenamiento con normalización finalizado.")

# 5. Evaluación (R^2)
Y_pred_test_scaled_neg = model_scaled_neg.predict(X_test_scaled_neg)
r2_scaled_neg = r2_score(Y_test_scaled_neg.flatten(), Y_pred_test_scaled_neg.flatten())

print(f"\n### Coeficiente de Determinación (R^2) con Normalización: {r2_scaled_neg:.4f} ###")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error


def predecir_y_evaluar_top_cuadrantes_neg(mes_a_predecir_neg, df_historico_neg, modelo_neg, escalador_neg, df_datos_reales_neg):
    """
    Predice y evalúa el robo promedio a negocios para los 78 cuadrantes en un mes N.

    Args:
        mes_a_predecir_neg (int): El número del mes que se quiere predecir (ej. 13).
        df_historico (pd.DataFrame): El DataFrame con los promedios históricos (df_final).
        modelo (tf.keras.Model): El modelo de Keras entrenado (model_scaled).
        escalador (sklearn.preprocessing.MinMaxScaler): El objeto MinMaxScaler.
        df_datos_reales (pd.DataFrame): DataFrame con la columna 'ROBOS_MES_N' real para ese mes N.

    Returns:
        tuple: (DataFrame del Top 10, R2, MAE)
    """

    print(f"--- Ejecutando Predicción y Evaluación para el Mes {mes_a_predecir_neg} ---")

    # 1. Definir meses históricos cíclicos
    mes_n_menos_1_neg = (mes_a_predecir_neg - 1)
    mes_n_menos_2_neg = (mes_a_predecir_neg - 2)
    mes_anterior_ciclo_neg = (mes_n_menos_1_neg - 1) % 12 + 1
    mes_anterior_2_ciclo_neg = (mes_n_menos_2_neg - 1) % 12 + 1

    # 2. Obtener los 78 datos de entrada de los meses N-1 y N-2
    df_n_menos_1_neg = df_historico_neg[df_historico_neg['MES_N'] == mes_anterior_ciclo_neg].copy()
    df_n_menos_2_neg = df_historico_neg[df_historico_neg['MES_N'] == mes_anterior_2_ciclo_neg].copy()

    # 3. Construir el DataFrame de Entrada (78 filas)
    df_prediccion_neg = df_historico_neg[df_historico_neg['MES_N'] == 1].copy()

    # Asegurar orden y rellenar las columnas de entrada
    df_prediccion_neg.sort_values(by='CUADRANTE', inplace=True)
    df_n_menos_1_neg.sort_values(by='CUADRANTE', inplace=True)
    df_n_menos_2_neg.sort_values(by='CUADRANTE', inplace=True)

    df_prediccion_neg['ROBOS_MES_N_MENOS_1'] = df_n_menos_1_neg['ROBOS_MES_N'].values
    df_prediccion_neg['ROBOS_MES_N_MENOS_2'] = df_n_menos_2_neg['ROBOS_MES_N'].values

    # 4. Preparar, Escalar y Aplanar la matriz X (312 entradas)
    X_pred_raw_neg = df_prediccion_neg[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']].values
    X_con_dummy_neg = np.hstack((X_pred_raw_neg, np.zeros((X_pred_raw_neg.shape[0], 1))))
    X_pred_escalado_completo_neg = escalador_neg.transform(X_con_dummy_neg)
    X_pred_final_neg = X_pred_escalado_completo_neg[:, :4].flatten().reshape(1, -1)

    # 5. Realizar la Predicción (Y_pred_scaled)
    Y_pred_scaled_neg = modelo_neg.predict(X_pred_final_neg)

    # 6. Desescalar la Predicción (Y_pred_desescalado)
    Y_pred_full_neg = np.hstack((X_pred_raw_neg, Y_pred_scaled_neg.T))
    Y_pred_desescalado_neg = escalador_neg.inverse_transform(Y_pred_full_neg)[:, -1]

    # 7. Generar el Top N (DataFrame de resultados)
    df_resultado_neg = pd.DataFrame({
        'CUADRANTE': df_prediccion_neg['CUADRANTE'].values,
        'PREDICCION_ROBOS_MES_N': Y_pred_desescalado_neg,
        'MES_PREDICHO': mes_a_predecir_neg
    })

    # 8. EVALUACIÓN Y COMPARACIÓN con Datos Reales
    # Aseguramos que el orden de los datos reales coincida con la predicción (por CUADRANTE)
    df_datos_reales_neg.sort_values(by='CUADRANTE', inplace=True)
    Y_real_neg = df_datos_reales_neg['ROBOS_MES_N'].values

    # Unir predicción y real para la visualización del Top N
    df_resultado_neg['ROBOS_REALES_MES_N'] = Y_real_neg

    # Cálculo de Métricas
    r2_neg = r2_score(Y_real_neg, Y_pred_desescalado_neg)
    mae_neg = mean_absolute_error(Y_real_neg, Y_pred_desescalado_neg)

    # Generar Top 10 con la comparación
    top_10_neg = df_resultado_neg.sort_values(by='PREDICCION_ROBOS_MES_N', ascending=False).head(10).round(2)

    return top_10_neg, r2_neg, mae_neg

# Modelo de predicción de robos de Vehículos

In [ ]:
import pandas as pd
import numpy as np

# 1. Cargar el archivo de promedios
file_path_veh = "exportados/Robos de vehiculos/cuadrantes_robos_vehiculos_promedio.xlsx"
df_promedios_veh = pd.read_excel(file_path_veh)

# Renombrar columnas para facilitar el manejo
column_names_veh = {
    'CUADRANTE': 'CUADRANTE',
    'POBLACION': 'POBLACION'
}
for i in range(1, 13):
    column_names_veh[f'PROMEDIO DE ROBOS DE VEHICULOS MES {i}'] = f'ROBOS_MES_{i}'

df_promedios_veh.rename(columns=column_names_veh, inplace=True)

# 2. Desapilar (Unpivot) la tabla para crear las 936 filas
# La columna 'POBLACION' se mantiene fija para cada cuadrante.
df_long_veh = df_promedios_veh.melt(
    id_vars=['CUADRANTE', 'POBLACION'],
    value_vars=[f'ROBOS_MES_{i}' for i in range(1, 13)],
    var_name='MES_NOMBRE',
    value_name='ROBOS_MES_N' # Este es el valor que queremos predecir (Y)
)

# 3. Extraer el número de mes (MES_N)
df_long_veh['MES_N'] = df_long_veh['MES_NOMBRE'].str.extract(r'(\d+)').astype(int)
df_long_veh.drop(columns=['MES_NOMBRE'], inplace=True)

# 4. Ordenar la tabla por CUADRANTE y MES_N
df_long_veh.sort_values(by=['MES_N', 'CUADRANTE'], inplace=True)
df_long_veh.reset_index(drop=True, inplace=True)

print("### 1. DataFrame de 936 Filas (Formato de Serie de Tiempo) ###")
print(df_long_veh.head(80).tail(5)) # Muestra los últimos del Mes 1
print(f"\nTotal de filas: {len(df_long_veh)}")
print("-" * 50)

In [ ]:
# 5. Crear la columna de Diciembre (MES 12) para usarla como N-1 del Mes 1
# Extraer los promedios de Diciembre (MES 12)
df_diciembre_veh = df_long_veh[df_long_veh['MES_N'] == 12].copy()
df_diciembre_veh.rename(columns={'ROBOS_MES_N': 'ROBOS_DIC'}, inplace=True)
df_diciembre_veh = df_diciembre_veh[['CUADRANTE', 'ROBOS_DIC']]

# Extraer los promedios de Noviembre (MES 11)
df_noviembre_veh = df_long_veh[df_long_veh['MES_N'] == 11].copy()
df_noviembre_veh.rename(columns={'ROBOS_MES_N': 'ROBOS_NOV'}, inplace=True)
df_noviembre_veh = df_noviembre_veh[['CUADRANTE', 'ROBOS_NOV']]

# 6. Crear las columnas N-1 y N-2 usando 'shift' agrupado
df_long_veh['ROBOS_MES_N_MENOS_1'] = df_long_veh.groupby('CUADRANTE')['ROBOS_MES_N'].shift(1)
df_long_veh['ROBOS_MES_N_MENOS_2'] = df_long_veh.groupby('CUADRANTE')['ROBOS_MES_N'].shift(2)

# 7. Imputar Enero (MES_N = 1 y 2) con los valores de Diciembre y Noviembre
# El Mes 1 (Enero) toma N-1 de Diciembre y N-2 de Noviembre.
df_long_veh = df_long_veh.merge(df_diciembre_veh, on='CUADRANTE', how='left')
df_long_veh.loc[df_long_veh['MES_N'] == 1, 'ROBOS_MES_N_MENOS_1'] = df_long_veh.loc[df_long_veh['MES_N'] == 1, 'ROBOS_DIC']

df_long_veh = df_long_veh.merge(df_noviembre_veh, on='CUADRANTE', how='left')
df_long_veh.loc[df_long_veh['MES_N'] == 1, 'ROBOS_MES_N_MENOS_2'] = df_long_veh.loc[df_long_veh['MES_N'] == 1, 'ROBOS_NOV']

# El Mes 2 (Febrero) toma N-2 de Diciembre
df_long_veh.loc[df_long_veh['MES_N'] == 2, 'ROBOS_MES_N_MENOS_2'] = df_long_veh.loc[df_long_veh['MES_N'] == 2, 'ROBOS_DIC']

df_final_veh = df_long_veh.drop(columns=['ROBOS_DIC', 'ROBOS_NOV'])

print("### 2. DataFrame Final con Variables N-1 y N-2 (Primeros 5 cuadrantes) ###")
print(df_final_veh.head(5))
print("-" * 50)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler # <-- para normalizar

# 1. Separar la tabla para aplicar el escalador
df_scale_veh = df_final_veh[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2', 'ROBOS_MES_N']].copy()

# 2. Aplicar el escalador (entrenado SOLO con los datos de entrada X)
# El escalador debe tratar todas las variables por separado
scaler_veh = MinMaxScaler()
df_scaled_values_veh = scaler_veh.fit_transform(df_scale_veh)
df_scaled_veh = pd.DataFrame(df_scaled_values_veh, columns=df_scale_veh.columns)
df_scaled_veh['MES_N'] = df_final_veh['MES_N'].values # Añadir el MES_N de vuelta

# 3. Estructurar X y Y para Keras (Múltiples Salidas) con datos escalados
X_features_scaled_veh = df_scaled_veh[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']]
Y_output_scaled_veh = df_scaled_veh['ROBOS_MES_N']

# Agrupar los datos escalados
X_data_scaled_veh = X_features_scaled_veh.groupby(df_scaled_veh['MES_N']).apply(lambda x: x.values.flatten()).tolist()
Y_data_scaled_veh = Y_output_scaled_veh.groupby(df_scaled_veh['MES_N']).apply(lambda x: x.values).tolist()

X_scaled_veh = np.array(X_data_scaled_veh)
Y_scaled_veh = np.array(Y_data_scaled_veh)

# Dividir para entrenamiento y prueba (mantenemos test_size=0.15 por ahora)
X_train_scaled_veh, X_test_scaled_veh, Y_train_scaled_veh, Y_test_scaled_veh = train_test_split(
    X_scaled_veh, Y_scaled_veh, test_size=0.15, shuffle=False
)

input_dim_veh = X_scaled_veh.shape[1]
output_dim_veh = Y_scaled_veh.shape[1]

# 4. Reconstruir y Entrenar el Modelo Keras (con datos escalados)
# Se reduce la tasa de aprendizaje (learning rate) como medida de precaución adicional
model_scaled_veh = Sequential([
    Dense(512, activation='relu', input_shape=(input_dim_veh,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim_veh, activation='linear')
])

model_scaled_veh.compile(optimizer=Adam(learning_rate=0.0005), loss='mse') # Reducción de LR
early_stop_veh = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

print("### Entrenamiento del Modelo Keras (Datos Normalizados) ###")
model_scaled_veh.fit(
    X_train_scaled_veh, Y_train_scaled_veh,
    epochs=200, # Aumentamos épocas por si acaso
    batch_size=2,
    validation_data=(X_test_scaled_veh, Y_test_scaled_veh),
    callbacks=[early_stop_veh],
    verbose=0
)
print("Entrenamiento con normalización finalizado.")

# 5. Evaluación (R^2)
Y_pred_test_scaled_veh = model_scaled_veh.predict(X_test_scaled_veh)
r2_scaled_veh = r2_score(Y_test_scaled_veh.flatten(), Y_pred_test_scaled_veh.flatten())

print(f"\n### Coeficiente de Determinación (R^2) con Normalización: {r2_scaled_veh:.4f} ###")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error


def predecir_y_evaluar_top_cuadrantes_veh(mes_a_predecir_veh, df_historico_veh, modelo_veh, escalador_veh, df_datos_reales_veh):
    """
    Predice y evalúa el robo promedio de vehículos para los 78 cuadrantes en un mes N.

    Args:
        mes_a_predecir_veh (int): El número del mes que se quiere predecir (ej. 13).
        df_historico (pd.DataFrame): El DataFrame con los promedios históricos (df_final).
        modelo (tf.keras.Model): El modelo de Keras entrenado (model_scaled).
        escalador (sklearn.preprocessing.MinMaxScaler): El objeto MinMaxScaler.
        df_datos_reales (pd.DataFrame): DataFrame con la columna 'ROBOS_MES_N' real para ese mes N.

    Returns:
        tuple: (DataFrame del Top 10, R2, MAE)
    """

    print(f"--- Ejecutando Predicción y Evaluación para el Mes {mes_a_predecir_veh} ---")

    # 1. Definir meses históricos cíclicos
    mes_n_menos_1_veh = (mes_a_predecir_veh - 1)
    mes_n_menos_2_veh = (mes_a_predecir_veh - 2)
    mes_anterior_ciclo_veh = (mes_n_menos_1_veh - 1) % 12 + 1
    mes_anterior_2_ciclo_veh = (mes_n_menos_2_veh - 1) % 12 + 1

    # 2. Obtener los 78 datos de entrada de los meses N-1 y N-2
    df_n_menos_1_veh = df_historico_veh[df_historico_veh['MES_N'] == mes_anterior_ciclo_veh].copy()
    df_n_menos_2_veh = df_historico_veh[df_historico_veh['MES_N'] == mes_anterior_2_ciclo_veh].copy()
    # 3. Construir el DataFrame de Entrada (78 filas)
    df_prediccion_veh = df_historico_veh[df_historico_veh['MES_N'] == 1].copy()

    # Asegurar orden y rellenar las columnas de entrada
    df_prediccion_veh.sort_values(by='CUADRANTE', inplace=True)
    df_n_menos_1_veh.sort_values(by='CUADRANTE', inplace=True)
    df_n_menos_2_veh.sort_values(by='CUADRANTE', inplace=True)

    df_prediccion_veh['ROBOS_MES_N_MENOS_1'] = df_n_menos_1_veh['ROBOS_MES_N'].values
    df_prediccion_veh['ROBOS_MES_N_MENOS_2'] = df_n_menos_2_veh['ROBOS_MES_N'].values

    # 4. Preparar, Escalar y Aplanar la matriz X (312 entradas)
    X_pred_raw_veh = df_prediccion_veh[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']].values
    X_con_dummy_veh = np.hstack((X_pred_raw_veh, np.zeros((X_pred_raw_veh.shape[0], 1))))
    X_pred_escalado_completo_veh = escalador_veh.transform(X_con_dummy_veh)
    X_pred_final_veh = X_pred_escalado_completo_veh[:, :4].flatten().reshape(1, -1)

    # 5. Realizar la Predicción (Y_pred_scaled)
    Y_pred_scaled_veh = modelo_veh.predict(X_pred_final_veh)

    # 6. Desescalar la Predicción (Y_pred_desescalado)
    Y_pred_full_veh = np.hstack((X_pred_raw_veh, Y_pred_scaled_veh.T))
    Y_pred_desescalado_veh = escalador_veh.inverse_transform(Y_pred_full_veh)[:, -1]
    # 7. Generar el Top N (DataFrame de resultados)
    df_resultado_veh = pd.DataFrame({
        'CUADRANTE': df_prediccion_veh['CUADRANTE'].values,
        'PREDICCION_ROBOS_MES_N': Y_pred_desescalado_veh,
        'MES_PREDICHO': mes_a_predecir_veh
    })

    # 8. EVALUACIÓN Y COMPARACIÓN con Datos Reales
    # Aseguramos que el orden de los datos reales coincida con la predicción (por CUADRANTE)
    df_datos_reales_veh.sort_values(by='CUADRANTE', inplace=True)
    Y_real_veh = df_datos_reales_veh['ROBOS_MES_N'].values

    # Unir predicción y real para la visualización del Top N
    df_resultado_veh['ROBOS_REALES_MES_N'] = Y_real_veh

    # Cálculo de Métricas
    r2_veh = r2_score(Y_real_veh, Y_pred_desescalado_veh)
    mae_veh = mean_absolute_error(Y_real_veh, Y_pred_desescalado_veh)
    # Generar Top 10 con la comparación
    top_10_veh = df_resultado_veh.sort_values(by='PREDICCION_ROBOS_MES_N', ascending=False).head(10).round(2)

    return top_10_veh, r2_veh, mae_veh